In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_normal', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Dropout(rate=0.3)(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Dropout(rate=0.3)(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
dropout_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
from keras.optimizers import RMSprop
dropout_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

In [8]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('dropout_training.log')

In [9]:
history = dropout_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 65s 1ms/step - loss: 0.4394 - acc: 0.8585 - val_loss: 0.0924 - val_acc: 0.9714
Epoch 2/20
50000/50000 [==============================] - 64s 1ms/step - loss: 0.1231 - acc: 0.9638 - val_loss: 0.0612 - val_acc: 0.9813
Epoch 3/20
50000/50000 [==============================] - 65s 1ms/step - loss: 0.0828 - acc: 0.9756 - val_loss: 0.0561 - val_acc: 0.9831
Epoch 4/20
50000/50000 [==============================] - 67s 1ms/step - loss: 0.0655 - acc: 0.9809 - val_loss: 0.0510 - val_acc: 0.9860
Epoch 5/20
50000/50000 [==============================] - 64s 1ms/step - loss: 0.0542 - acc: 0.9846 - val_loss: 0.0494 - val_acc: 0.9867
Epoch 6/20
50000/50000 [==============================] - 64s 1ms/step - loss: 0.0474 - acc: 0.9863 - val_loss: 0.0480 - val_acc: 0.9875
Epoch 7/20
50000/50000 [==============================] - 63s 1ms/step - loss: 0

In [10]:
(loss, accuracy) = dropout_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 6s 643us/step
accuracy: 98.97%


In [11]:
import pandas as pd
df = pd.read_csv('dropout_training.log')
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.85850  0.439379   0.9714  0.092388
1       1  0.96384  0.123074   0.9813  0.061244
2       2  0.97560  0.082817   0.9831  0.056116
3       3  0.98088  0.065493   0.9860  0.051014
4       4  0.98464  0.054220   0.9867  0.049374
5       5  0.98632  0.047377   0.9875  0.047976
6       6  0.98754  0.042756   0.9855  0.061049
7       7  0.98906  0.038734   0.9874  0.050419
8       8  0.99028  0.034596   0.9885  0.048295
9       9  0.99086  0.032528   0.9895  0.047911
10     10  0.99124  0.029571   0.9884  0.050683
11     11  0.99140  0.028860   0.9878  0.055412
12     12  0.99210  0.027432   0.9895  0.057079
13     13  0.99244  0.027656   0.9890  0.052614
14     14  0.99244  0.026229   0.9882  0.056848
15     15  0.99296  0.024921   0.9878  0.069219
16     16  0.99302  0.025869   0.9861  0.054103
17     17  0.99306  0.025731   0.9881  0.064482
18     18  0.99360  0.023096   0.9885  0.061186
19     19  0.99310  0.025815   0.9887  0